## Collecting protocol fees across Balancer core pools on all networks
- Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
- Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [1]:
import datetime
import pandas as pd
from dotenv import load_dotenv
load_dotenv()
from web3 import Web3

from notebooks import get_block_by_ts
from notebooks import get_twap_bpt_price
from config_file import CORE_POOLS
from config_file import Chains
from config_file import web3_instances

TARGET_BLOCKS = {}
# ARBITRUM
timestamp_now = 1695952610
timestamp_2_weeks_ago = timestamp_now - (2 * 7 * 24 * 60 * 60)
datetime_now = datetime.datetime.fromtimestamp(timestamp_now)
datetime_2_weeks_ago = datetime.datetime.fromtimestamp(timestamp_2_weeks_ago)

arb_block = get_block_by_ts(timestamp_now, "arbitrum")  # 18 August 2023
TARGET_BLOCKS[Chains.ARBITRUM.value] = arb_block
# Given Arb block time, we want to look back 2 weeks:
arb_block_2_weeks_ago = get_block_by_ts(timestamp_2_weeks_ago, "arbitrum")
# MAINNET
mainnet_block = get_block_by_ts(timestamp_now, "mainnet")
TARGET_BLOCKS[Chains.MAINNET.value] = mainnet_block
# Given mainnet block time, we want to look back 2 weeks:
mainnet_block_2_weeks_ago = get_block_by_ts(timestamp_2_weeks_ago, "mainnet")
# BASE
base_block = get_block_by_ts(timestamp_now, "base")
TARGET_BLOCKS[Chains.BASE.value] = base_block
# Given base block time, we want to look back 2 weeks:
base_block_2_weeks_ago = get_block_by_ts(timestamp_2_weeks_ago, "base")

# POLYGON
poly_block = get_block_by_ts(timestamp_now, "polygon")
TARGET_BLOCKS[Chains.POLYGON.value] = poly_block
# Given polygon block time, we want to look back 2 weeks:
poly_block_2_weeks_ago = get_block_by_ts(timestamp_2_weeks_ago, "polygon")

# GNOSIS
gnosis_block = get_block_by_ts(timestamp_now, "gnosis")
TARGET_BLOCKS[Chains.GNOSIS.value] = gnosis_block
# Given gnosis block time, we want to look back 2 weeks:
gnosis_block_2_weeks_ago = get_block_by_ts(timestamp_2_weeks_ago, "gnosis")

# Avalanche
avax_block = get_block_by_ts(timestamp_now, "avalanche")
TARGET_BLOCKS[Chains.AVALANCHE.value] = avax_block
# Given avalanche block time, we want to look back 2 weeks:
avax_block_2_weeks_ago = get_block_by_ts(timestamp_2_weeks_ago, "avalanche")

bpt_twap_prices = {}
gnosis_bpt_twap_prices = {}
for chain in Chains:
    print(f"Collecting bpt prices for {chain.value}")
    pools = CORE_POOLS.get(chain.value, None)
    if pools is None:
        continue
    for core_pool in pools:
        bpt_twap_prices[core_pool] = get_twap_bpt_price(
            core_pool, chain.value, getattr(web3_instances, chain.value), start_date=datetime.datetime.fromtimestamp(timestamp_now), block_number=TARGET_BLOCKS[chain.value]
        )
print(bpt_twap_prices)

{'0xf0ad209e2e969eaaa8c882aac71f02d8a047d5c2000200000000000000000b49': Decimal('0.5280424513237304447412142192'), '0xee278d943584dd8640eaf4cc6c7a5c80c0073e85000200000000000000000bc7': Decimal('0.5258587006211421026950010597'), '0x93d199263632a4ef4bb438f1feb99e57b4b5f0bd0000000000000000000005c2': Decimal('1615.639222768882312511327769'), '0x1e19cf2d73a72ef1332c882f20534b6519be0276000200000000000000000112': Decimal('1655.742499099692472457334460'), '0xe7e2c68d3b13d905bbb636709cf4dfd21076b9d20000000000000000000005ca': Decimal('1583.635774961356404658467626'), '0xf7a826d47c8e02835d94fb0aa40f0cc9505cb1340002000000000000000005e0': Decimal('1612.932033705533336612464332'), '0xf16aee6a71af1a9bc8f56975a4c2705ca7a782bc0002000000000000000004bb': Decimal('27.42670529040436556407878377'), '0xb08885e6026bab4333a80024ec25a1a3e1ff2b8a000200000000000000000445': Decimal('1628.179323995485920040465830'), '0x8353157092ed8be69a9df8f95af097bbf33cb2af0000000000000000000005d9': Decimal('0.98590601831121297567

## Fetching data from the Balancer subgraphs

In [ ]:
from notebooks import get_balancer_pool_snapshots
from notebooks.fees_and_bribs.constants import BASE_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import POLYGON_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import ARB_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import MAINNET_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import GNOSIS_BALANCER_GRAPH_URL
from notebooks.fees_and_bribs.constants import AVALANCHE_BALANCER_GRAPH_URL
from typing import Dict

arbi_pool_snapshots_now = get_balancer_pool_snapshots(arb_block, ARB_BALANCER_GRAPH_URL)
arbi_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(arb_block_2_weeks_ago), ARB_BALANCER_GRAPH_URL)

mainnet_pool_snapshots_now = get_balancer_pool_snapshots(mainnet_block, MAINNET_BALANCER_GRAPH_URL)
mainnet_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(mainnet_block_2_weeks_ago),
                                                                 MAINNET_BALANCER_GRAPH_URL)

polygon_pool_snapshots_now = get_balancer_pool_snapshots(poly_block, POLYGON_BALANCER_GRAPH_URL)
polygon_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(poly_block_2_weeks_ago),
                                                                 POLYGON_BALANCER_GRAPH_URL)

base_pool_snapshots_now = get_balancer_pool_snapshots(base_block, BASE_BALANCER_GRAPH_URL)
base_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(base_block_2_weeks_ago),
                                                              BASE_BALANCER_GRAPH_URL)

gnosis_pool_snapshots_now = get_balancer_pool_snapshots(gnosis_block, GNOSIS_BALANCER_GRAPH_URL)
gnosis_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(gnosis_block_2_weeks_ago),
                                                                GNOSIS_BALANCER_GRAPH_URL)

avax_pool_snapshots_now = get_balancer_pool_snapshots(avax_block, AVALANCHE_BALANCER_GRAPH_URL)
avax_pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(avax_block_2_weeks_ago),
                                                              AVALANCHE_BALANCER_GRAPH_URL)

## Extract fee data for CORE pools:


In [ ]:
from collections import defaultdict
from decimal import Decimal
from notebooks import fetch_token_price_balgql


def collect_fee_info(pools: list[str], chain: str, pools_now: list[dict], pools_shifted: list[dict],
                     start_date: datetime.datetime) -> tuple[dict, dict]:
    # Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
    # and then summing across all pools
    fees = {}
    token_fees = defaultdict(list)
    for pool in pools:
        current_fees_snapshots = [x for x in pools_now if x['pool']['id'] == pool]
        current_fees_snapshots.sort(key=lambda x: x['timestamp'], reverse=True)
        fees_2_weeks_ago = [x for x in pools_shifted if x['pool']['id'] == pool]
        fees_2_weeks_ago.sort(key=lambda x: x['timestamp'], reverse=True)
        # Take first element of list, which is the most recent snapshot
        if not current_fees_snapshots or not fees_2_weeks_ago:
            continue
        pool_snapshot_now = current_fees_snapshots[0]
        pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
        # Calculate fees
        pool_fee = float(pool_snapshot_now['protocolFee']) - float(pool_snapshot_2_weeks_ago['protocolFee'])
        pool_swap_fee = float(pool_snapshot_now['swapFees']) - float(pool_snapshot_2_weeks_ago['swapFees'])
        # Now we need to collect token fee info. Let's start with BPT tokens, which is Balancer pool token. Notice,
        # That totalProtocolFeePaidInBPT can be null, so we need to check for that
        bpt_token_fee = 0
        bpt_price_usd = bpt_twap_prices[chain][pool]
        if bpt_price_usd is None:
            bpt_price_usd = 0
        if pool_snapshot_now['pool']['totalProtocolFeePaidInBPT'] is not None and pool_snapshot_2_weeks_ago['pool'][
            'totalProtocolFeePaidInBPT'] is not None:
            bpt_token_fee = float(pool_snapshot_now['pool']['totalProtocolFeePaidInBPT']) - float(
                pool_snapshot_2_weeks_ago['pool']['totalProtocolFeePaidInBPT'])
            token_fees[pool_snapshot_now['pool']['id']].append({
                'symbol': pool_snapshot_now['pool']['symbol'],
                'token': pool_snapshot_now['pool']['symbol'],
                'token_fee': bpt_token_fee,
                'token_price': bpt_price_usd,
                'token_fee_in_usd': Decimal(bpt_token_fee) * bpt_price_usd,
                'token_addr': pool_snapshot_now['pool']['address'],
                'time_from': datetime_2_weeks_ago,
                'time_to': datetime_now,
                'chain': chain,
            })
        # Now collect fee info about fees paid in pool tokens. Pool tokens fee info is in pool.tokens dictionary. This will be separate dictionary
        else:
            bpt_price_usd = 0
            for token_data in pool_snapshot_now['pool']['tokens']:
                token_data_2_weeks_ago = \
                    [t for t in pool_snapshot_2_weeks_ago['pool']['tokens'] if t['address'] == token_data['address']][0]
                token_fee = float(token_data.get('paidProtocolFees', None)) - float(
                    token_data_2_weeks_ago.get('paidProtocolFees', None))
                # Get twap token price from CoinGecko
                token_price = fetch_token_price_balgql(token_data['address'], chain, start_date) or 0
                token_fees[pool_snapshot_now['pool']['id']].append({
                    'symbol': pool_snapshot_now['pool']['symbol'],
                    'token': token_data['symbol'],
                    'token_fee': token_fee,
                    'token_price': token_price,
                    'token_fee_in_usd': Decimal(token_fee) * token_price if token_price is not None else 0,
                    'token_addr': token_data['address'],
                    'time_from': datetime_2_weeks_ago,
                    'time_to': datetime_now,
                    'chain': chain,
                })
        # Calculate non-BPT fees in USD
        fees[pool_snapshot_now['pool']['id']] = {
            'symbol': pool_snapshot_now['pool']['symbol'],
            'pool_fee': round(pool_fee, 2),
            'swap_fee': round(pool_swap_fee, 2),
            'bpt_token_fee': round(bpt_token_fee, 2),
            # Get fee in USD by multiplying bpt_token_fee by price of BPT token taken from twap_bpt_price
            'bpt_token_fee_in_usd': round(Decimal(bpt_token_fee) * bpt_price_usd, 2),
            'token_fees_in_usd': round(sum([x['token_fee_in_usd'] for x in
                                            token_fees[
                                                pool_snapshot_now['pool']['symbol']]]) if bpt_price_usd == 0 else 0, 2),
            'time_from': datetime_2_weeks_ago,
            'time_to': datetime_now,
            'chain': chain,
            'token_fees': token_fees[pool_snapshot_now['pool']['symbol']],
            'pool_addr': pool_snapshot_now['pool']['address'],
        }
    return fees


arb_fees = collect_fee_info(CORE_POOLS.arbitrum, 'arbitrum', arbi_pool_snapshots_now,
                            arbi_pool_snapshots_2_weeks_ago, datetime_now)
mainnet_fees = collect_fee_info(CORE_POOLS.mainnet, 'mainnet', mainnet_pool_snapshots_now,
                                mainnet_pool_snapshots_2_weeks_ago, datetime_now)
polygon_fees = collect_fee_info(CORE_POOLS.polygon, 'polygon', polygon_pool_snapshots_now,
                                polygon_pool_snapshots_2_weeks_ago, datetime_now)
base_fees = collect_fee_info(CORE_POOLS.base, 'base', base_pool_snapshots_now, base_pool_snapshots_2_weeks_ago,
                             datetime_now)
# gnosis_fees = collect_fee_info(CORE_POOLS.gnosis, 'gnosis', gnosis_pool_snapshots_now,
#                                gnosis_pool_snapshots_2_weeks_ago, gnosis_datetime_now)
avax_fees = collect_fee_info(CORE_POOLS.avalanche, 'avalanche', avax_pool_snapshots_now,
                             avax_pool_snapshots_2_weeks_ago, datetime_now)
# Convert to dataframe, sort by chain and pool fee
joint_fees = {**arb_fees, **mainnet_fees, **polygon_fees, **base_fees, **avax_fees}
joint_fees_df = pd.DataFrame.from_dict(joint_fees, orient='index')

In [ ]:
# Remove `token_fees` field from dataframe, as it's too big
joint_fees_df_copy = joint_fees_df.drop(columns=['token_fees'])
# Display all rows in dataframe
pd.set_option('display.max_rows', 1000)
joint_fees_df_copy.sort_values(by=['chain', 'pool_fee'], ascending=False)

## Now let's calculate bribes paid to the pools

In [ ]:
from notebooks import calculate_aura_vebal_share

aura_vebal_share = calculate_aura_vebal_share(web3_instances.mainnet, mainnet_block)

# Bribes are split per chain and per pool, with each pool getting a share of the bribe proportional to its share of fees
# paid by all pools on that chain. For example, if pool A paid 10% of all fees on Arbitrum, it will get 10% of the bribes. That 10% will be distributed between aura and vebal, proportional to their share of the bribe.
FEE = Decimal(0.5)  # 50% goes to fees


# Let's calculate share of fees paid by each pool on each chain
def calc_and_split_bribes(fees: Dict, chain: str, fees_to_distribute: Decimal) -> Dict[str, Dict]:
    pool_bribs = {}
    # Calculate pool share in fees
    total_fees = sum([data['pool_fee'] for pool, data in fees.items()])
    for pool, data in fees.items():
        pool_fees = data['bpt_token_fee_in_usd'] + data['token_fees_in_usd']
        pool_share = pool_fees / Decimal(total_fees)
        # If aura bribes is less than 500 USDC, we pay all bribes to balancer
        aura_bribes = round(pool_share * fees_to_distribute * aura_vebal_share * FEE, 2)
        if aura_bribes <= Decimal(500):
            aura_bribes = Decimal(0)
            bal_bribes = round(pool_share * fees_to_distribute * FEE, 2)
        else:
            bal_bribes = round(pool_share * fees_to_distribute * (1 - aura_vebal_share) * FEE, 2)
        # Split fees between aura and bal fees
        pool_bribs[pool] = {
            "symbol": data['symbol'],
            "chain": chain,
            "aura_bribes": aura_bribes,
            "bal_bribes": bal_bribes,
            "protocol_fees_collected": pool_fees,
            "pool_addr": data['pool_addr'],
        }
    return pool_bribs


# TODO: Move to constants
mainnet_bribes = calc_and_split_bribes(mainnet_fees, 'mainnet', Decimal(112945.77))
arb_bribes = calc_and_split_bribes(arb_fees, 'arbitrum', Decimal(39481.82))
polygon_bribes = calc_and_split_bribes(polygon_fees, 'polygon', Decimal(6592.87))
base_bribes = calc_and_split_bribes(base_fees, 'base', Decimal(10260.35))
# gnosis_bribes = calc_and_split_bribes(gnosis_fees, 'gnosis', Decimal(2503.21))
# avax_bribes = calc_and_split_bribes(avax_fees, 'avalanche', Decimal(10297))
# Convert to dataframe
# joint_bribes_data = {**arb_bribes, **mainnet_bribes, **polygon_bribes, **base_bribes, **gnosis_bribes, **avax_bribes}
joint_bribes_data = {**arb_bribes, **mainnet_bribes, **polygon_bribes, **base_bribes}
# Sort by chain:
joint_bribes_data = {k: v for k, v in sorted(joint_bribes_data.items(), key=lambda item: item[1]['chain'])}
joint_bribes_df = pd.DataFrame.from_dict(joint_bribes_data, orient='index')
# Sort by chain
# Dump into csv and prefix with dates
joint_bribes_df.to_csv(f'./output/bribes_{datetime_2_weeks_ago.date()}_{datetime_now.date()}.csv')
joint_bribes_df

In [ ]:
from notebooks import fetch_all_pools_info

mainnet_pools_info = fetch_all_pools_info('mainnet')
arb_pools_info = fetch_all_pools_info('arbitrum')
polygon_pools_info = fetch_all_pools_info('polygon')
base_pools_info = fetch_all_pools_info('base')
gnosis_pools_info = fetch_all_pools_info('gnosis')
avax_pools_info = fetch_all_pools_info('avalanche')

mainnet_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
                  mainnet_pools_info}
arb_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
              arb_pools_info}
polygon_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
                  polygon_pools_info}
base_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
               base_pools_info}
gnosis_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
                 gnosis_pools_info}
avax_gauges = {pool_info['address']: Web3.to_checksum_address(pool_info['gauge']['address']) for pool_info in
               avax_pools_info}
# Put it all into mapping per chain:
mapped_gauges = {
    'mainnet': mainnet_gauges,
    'arbitrum': arb_gauges,
    'polygon': polygon_gauges,
    'base': base_gauges,
    'gnosis': gnosis_gauges,
    'avalanche': avax_gauges,
}

In [ ]:
# Load csv sheet to compare with the data collected above:
bribes_df = pd.read_csv('compare_sheet.csv')
# To dict:
bribes_to_compare = bribes_df.to_dict(orient='records')
# Each dict in list needs target to be Web3.toChecksumAddress
bribes_csv_checksummed = []
for bribe in bribes_to_compare:
    bribes_csv_checksummed.append({
        'target': Web3.to_checksum_address(bribe['target']),
        'platform': bribe['platform'],
        'amount': bribe['amount'],
    })
# These are bribes paid de-facto, so we need to convert them to the same format as we have in our data
bribes_final_csv = {}
for bribe in bribes_csv_checksummed:
    if bribe['target'] not in bribes_final_csv:
        bribes_final_csv[bribe['target']] = {}
    bribes_final_csv[bribe['target']][bribe['platform']] = Decimal(bribe['amount'])

## Compare data from the sheet with the data we collected
What does table below represent:
- aura_automated_bribe: bribes paid to aura per pool calculated in this notebook
- aura_actual_bribe: actual bribes paid to aura per pool from the sheet
- aura_bribe_delta: difference between aura_automated_bribe and aura_actual_bribe
- aura_bribe_delta_%: difference between aura_automated_bribe and aura_actual_bribe in %

Same thing for balancer side:
- bal_automated_bribe: bribes paid to balancer per pool calculated in this notebook
- bal_actual_bribe: actual bribes paid to balancer per pool from the sheet
- bal_bribe_delta: difference between bal_automated_bribe and bal_actual_bribe
- bal_bribe_delta_%: difference between bal_automated_bribe and bal_actual_bribe in %

In [ ]:
# Now we need to compare the data we collected with the data from the sheet
# We need to compare bribes paid to each pool
joint_bribes_dict = joint_bribes_df.to_dict(orient='records')
bribes_delta = {}
for gauge, data in bribes_final_csv.items():
    for automated_bribe in joint_bribes_dict:
        filtered_gauges = mapped_gauges[automated_bribe['chain']]
        # If this condition is true, we can compare the bribes
        if filtered_gauges[automated_bribe['pool_addr']] == gauge:
            # Calculate delta between automated bribes and bribes from the sheet
            # Calculate delta between automated_bribe['aura_bribes'] and data['aura'] in %
            if automated_bribe['aura_bribes'] > data['aura']:
                aura_bribe_delta = automated_bribe['aura_bribes'] - data['aura']
                aura_bribe_delta_pct = (automated_bribe['aura_bribes'] - data['aura']) / data['aura'] * 100 if \
                    data['aura'] != 0 else 0
            else:
                aura_bribe_delta = data['aura'] - automated_bribe['aura_bribes']
                aura_bribe_delta_pct = (data['aura'] - automated_bribe['aura_bribes']) / data['aura'] * 100 if \
                    data['aura'] != 0 else 0
            if automated_bribe['bal_bribes'] > data['balancer']:
                bal_bribe_delta = automated_bribe['bal_bribes'] - data['balancer']
                bal_bribe_delta_pct = (automated_bribe['bal_bribes'] - data['balancer']) / data['balancer'] * 100 if \
                    data['balancer'] != 0 else 0
            else:
                bal_bribe_delta = data['balancer'] - automated_bribe['bal_bribes']
                bal_bribe_delta_pct = (data['balancer'] - automated_bribe['bal_bribes']) / data['balancer'] * 100 if \
                    data['balancer'] != 0 else 0
            bribes_delta[gauge] = {
                'symbol': automated_bribe['symbol'],
                'chain': automated_bribe['chain'],
                'aura_automated_bribe': automated_bribe['aura_bribes'],
                'aura_actual_bribe': round(data['aura'], 2),
                'aura_bribe_delta': round(aura_bribe_delta, 2),
                'aura_bribe_delta_%': f'{round(aura_bribe_delta_pct, 2)}%',
                'bal_automated_bribe': automated_bribe['bal_bribes'],
                'bal_actual_bribe': round(data['balancer'], 2),
                'bal_bribe_delta': round(bal_bribe_delta, 2),
                'bal_bribe_delta_%': f'{round(bal_bribe_delta_pct, 2)}%',
            }
# To df and print out without index:
bribes_delta_df = pd.DataFrame.from_dict(bribes_delta, orient='index')
bribes_delta_df.sort_values(by=['chain', 'aura_bribe_delta'], ascending=False)
# Also dump to csv
bribes_delta_df.to_csv(f'./output/bribes_delta_{datetime_2_weeks_ago.date()}_{datetime_now.date()}.csv')
bribes_delta_df